In [ ]:
import rasgoql

#### Helper Methods

In [ ]:
# Helper methods available prior to connection

print(rasgoql.__all__)
print(rasgoql.__version__)
rasgoql.open_docs()

### Connection Workflow

In [ ]:
# -- First Run Only --
# Set credentials for your DataWarehouse and save them in a .env file
creds = rasgoql.SnowflakeCredentials(
    account="",
    user="",
    password="",
    role="",
    warehouse="",
    database="",
    schema=""
)
optional_file_path = 'myfolder/'
creds.to_env(optional_file_path)

# or more simply:
creds.to_env()

In [ ]:
# Connect RQL to your Data Warehouse

# Setting creds here is redundant to the cell above
# This is what will be run in subsequent sessions after env 
# variables are set in the initial session
creds = rasgoql.SnowflakeCredentials.from_env()

rql = rasgoql.connect(creds)

print("rasgoQL connected!")

### Interact with DW Workflow

In [ ]:
# List all available tables in DW (pull into pandas)
rql.list_tables()

In [ ]:
# Run a custom query
rql.query("SELECT 'Hold onto your butts' AS free_form_sql ")

In [ ]:
# rasgoQL tries to help prevent the most basic SQL injection patterns
rql.query('SELECT top 10 *; TRUNCATE VERY_IMPORTANT_TABLE')

In [ ]:
# Pull results from a custom query into pandas (if it returns data)
tbl_df = rql.query_into_df("SELECT TOP 10 * FROM ABCD123")
tbl_df.head()

### Interact with Datasets Workflow

In [ ]:
# Allow rasgoQL to interact with an existing Dataset in your Data Warehouse
ds = rql.dataset(fqtn='RASGOLOCAL.PUBLIC.ABCD123')
ds

In [ ]:
# Preview the Dataset in pandas
ds.preview()

# Or pull the whole Dataset down
#df = ds.to_df()
#df.head()

In [ ]:
# Preview columns & data types in table
ds.get_schema()

In [ ]:
# Get table's DDL
ds.sql()

### Apply Transforms Workflow

In [ ]:
# Browse available rasgoQL transform templates
templates = rql.list_transforms()
print(templates)

# interact with the list
for t in templates:
    if t.name == 'join':
        print(t.define())

In [ ]:
# Define a specific transform
print(rql.define_transform('apply'))

In [ ]:
# Start with an existing Dataset
ds = rql.dataset(fqtn='RASGOLOCAL.PUBLIC.ABCD123')
print(ds)

# Create a SQL chain by applying Transform(s)
chn = tbl.cast(
    casts={
      'LOC_ID':'STRING',
      'DATE':'STRING'
    }
)
chn

In [ ]:
# Print SQL
print(chn.sql())

In [ ]:
# Fork existing chains and keep adding to them
# Fork 1
chn1 = chn.concat(concat_list=["STATE_NAME", "'some_str'", "'_F1'"])
print('Fork 1:')
print(chn1)

# Fork 2
chn2 = chn.cast(casts = {'DATE':'date'})
chn2 = chn2.datetrunc(dates = {'DATE_DATE':'month'})
print('Fork 2:')
print(chn2)

In [ ]:
# Print SQL
print('Fork 1:')
print(chn1.sql())

print('\n-----\n')

print('Fork 2:')
print(chn2.sql())

In [ ]:
# Try different render methods
print('\nRender CTE')
print(chn2.sql(render_method='SELECT'))

print('\nRender View')
print(chn2.sql(render_method='VIEW'))

In [ ]:
# Preview in pandas
chn2.preview()

In [ ]:
# Save the chain to your Data Warehouse as a view
new_view = chn2.save(table_type='view')
new_view

In [ ]:
# Print the view DDL SQL to confirm it's what we expect
print('\nDDL')
print(new_view.sql())

In [ ]:
# Coming Soon: Create a dbt-compliant YML file to run the transforms
#chn2.to_yml()